In [ ]:
import workflow.scripts.resources as res_module
import pandas as pd
import geopandas as gpd

In [ ]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis
store=f"data/store/resources_AB.h5"
datahandler=DataHandler(store)

# Set Required Args to Activate Modules

In [ ]:
# Iterate over provinces for both solar and wind resources
resource_types = ['wind'] # , 'solar'
provinces=['AB']  #'AB','SK','ON','NS'
for province_code in provinces:
    for resource_type in resource_types:
        required_args = {
            "config_file_path": 'config/config.yaml',
            "province_short_code": province_code,
            "resource_type": resource_type
        }
        
        # Create an instance of Resources and execute the module
        resources = res_module.Resources(**required_args)
        resources.execute_module()

# Temp tests

In [ ]:
CPCAD=pd.read_pickle('data/downloaded_data/lands/ProtectedConservedArea.pickle')

In [ ]:
datahandler.show_tree(store)

In [ ]:
boundary=datahandler.from_store('boundary')

In [ ]:
CPCAD=pd.read_pickle('data/downloaded_data/lands/ProtectedConservedArea_AB.pickle')

In [ ]:
CPCAD.explore('NAME_E')

In [ ]:
# Filter for Banff and Waterton Lakes National Parks
banff_and_waterton = CPCAD[
    (CPCAD['NAME_E'] == 'Banff National Park Of Canada') | 
    (CPCAD['NAME_E'] == 'Waterton Lakes National Park Of Canada')
]

# Check CRS and reproject if needed
if banff_and_waterton.crs.is_geographic:
    banff_and_waterton = banff_and_waterton.to_crs(epsg=3857)  # Reproject to meters for accurate buffering

# Apply the buffer (e.g., 10 km buffer)
buffered_boundary = banff_and_waterton.geometry.buffer(30000)  # Distance in meters

# Create a new GeoDataFrame with the buffered geometry
buffered_banff = gpd.GeoDataFrame(banff_and_waterton, geometry=buffered_boundary)

# Reproject back to the original CRS if needed
buffered_banff = buffered_banff.to_crs(CPCAD.crs)

# Plot the buffered area
buffered_banff.explore()


In [ ]:
datahandler.show_tree(store)

In [ ]:
cells=datahandler.from_store('cells')
cells.to_pickle('AB_temp_analysis/Cells_without_regulatory_buffer.pkl')

In [ ]:
cells.columns

In [ ]:
cells.explore('potential_capacity_wind',cmap='Blues',alpha=0.6)